<a href="https://colab.research.google.com/github/Ino54/MA_GreenAI-Practical-Experiments/blob/main/bloom_hardware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------- Requirements ----------
%%writefile requirements.txt
transformers
accelerate
datasets
evaluate
sacrebleu
codecarbon>=2.5,<3
pynvml>=11.5.0
psutil
numpy
pandas
huggingface_hub

Writing requirements.txt


In [ ]:
!pip -q install -r requirements.txt
# potenzielle Konflikte leise entfernen
!pip uninstall -y -q google-genai firebase-admin || true

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.6/517.6 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.38.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
# ---------- Hugging Face Login via Colab-Secret ----------
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(hf_token); print("Hugging Face Login erfolgreich!")
else:
    print("WARNUNG: Kein HF_TOKEN gefunden")

Hugging Face Login erfolgreich!


In [ ]:
# ---------- Drive mounten & Zielordner prüfen ----------
import os, shutil, time, pathlib, platform, gc, re, math, warnings, inspect
warnings.filterwarnings("ignore")

from google.colab import drive
MOUNTPOINT = "/content/drive"
already = os.path.isdir(os.path.join(MOUNTPOINT, "MyDrive"))
if not already and os.path.isdir(MOUNTPOINT) and os.listdir(MOUNTPOINT):
    backup = f"/content/drive_stale_{int(time.time())}"
    shutil.move(MOUNTPOINT, backup)
    os.makedirs(MOUNTPOINT, exist_ok=True)
drive.mount(MOUNTPOINT, force_remount=(not already))

work_dir = "/content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_hardware"
if not os.path.isdir(work_dir):
    raise FileNotFoundError(
        f"Zielordner fehlt: {work_dir}\n"
        "Bitte diesen Ordner manuell in Google Drive anlegen und das Notebook erneut starten."
    )
os.chdir(work_dir)
project_dir = work_dir
print("Arbeitsordner:", os.getcwd())

Mounted at /content/drive
Arbeitsordner: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_hardware


In [ ]:
# ---------- Imports & Device ----------
import numpy as np, pandas as pd
import torch, psutil
from contextlib import nullcontext
from types import SimpleNamespace
from datasets import load_dataset
import evaluate

from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from codecarbon import EmissionsTracker

# PyTorch Runtimes (Performance)
import os as _os
_os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # weniger Fragmentierung

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    vram_total_gb = torch.cuda.get_device_properties(0).total_memory/(1024**3)
    torch.backends.cuda.matmul.allow_tf32 = True  # T4 ignoriert TF32 (ok)
    torch.backends.cudnn.benchmark = True
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass
else:
    gpu_name = "CPU"; vram_total_gb = 0.0
print(f"Device: {device} | GPU: {gpu_name} | VRAM={vram_total_gb:.1f} GB | Torch {torch.__version__} | Py {platform.python_version()}")

RESULT_BASENAME = "bloom_hardware"

Device: cuda | GPU: NVIDIA A100-SXM4-80GB | VRAM=79.3 GB | Torch 2.8.0+cu126 | Py 3.12.11


In [ ]:
# ---------- Flags für Hardware-Optimierung ----------
APPLY_TORCH_COMPILE     = False    # für kurze Läufe meist AUS; bei langen Läufen True setzen
PINNED_MEM              = True     # Host-Puffer pinnen
NON_BLOCKING            = True     # non_blocking Transfers zur GPU

In [ ]:
# ---------- CodeCarbon-Helfer ----------
def _cleanup_cc_locks():
    for p in [
        "/tmp/.codecarbon.lock",
        _os.path.expanduser("~/.codecarbon/codecarbon.lock"),
        "/content/.codecarbon/codecarbon.lock",
    ]:
        try:
            if _os.path.exists(p):
                _os.remove(p)
                print(f"[CodeCarbon] Lock entfernt: {p}")
        except Exception:
            pass

_os.environ["CODECARBON_CACHE_DIR"] = f"/content/.cc_cache_hw_{int(time.time())}"

def _cc_supported_kwargs():
    base = dict(log_level="error", output_dir=".", measure_power_secs=1, tracking_mode="process")
    try:
        params = inspect.signature(EmissionsTracker.__init__).parameters
        if "allow_multiple_runs" in params: base["allow_multiple_runs"] = True
        if "cloud_provider" in params:      base["cloud_provider"] = "google"
        if "cloud_region" in params:        base["cloud_region"]   = "europe-west10"
        if "country_iso_code" in params:    base["country_iso_code"] = "DEU"
    except Exception:
        pass
    return base

def make_trk(name, out):
    _cleanup_cc_locks()
    return EmissionsTracker(project_name=name, output_file=out, **_cc_supported_kwargs())

def start(tr):
    try:
        tr.start(); return True
    except Exception:
        _cleanup_cc_locks()
        try:
            tr.start(); return True
        except Exception:
            return False

def stop(tr, st):
    if not st:
        return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
    try:
        return tr.stop()
    except Exception:
        return SimpleNamespace(energy_consumed=0.0, emissions=0.0)

def unpack(em):
    if hasattr(em, "energy_consumed") and hasattr(em, "emissions"):
        try: return float(em.energy_consumed), float(em.emissions)
        except: return 0.0, 0.0
    if isinstance(em, dict):
        e = em.get("energy_consumed", 0.0)
        c = em.get("emissions", em.get("emissions_kg", 0.0))
        try: return float(e), float(c)
        except: return 0.0, 0.0
    try: return 0.0, float(em)
    except: return 0.0, 0.0

def read_energy(path):
    try:
        if not _os.path.exists(path): return 0.0
        df = pd.read_csv(path)
        for c in ["energy_consumed","energy_consumed_kwh","energy_consumed (kWh)","energy (kWh)"]:
            if c in df.columns: return float(df[c].iloc[-1])
        for c in df.columns:
            if "energy" in c.lower() and "kwh" in c.lower():
                return float(df[c].iloc[-1])
    except Exception:
        pass
    return 0.0

def measure(phase, fn, prefix):
    logfile = _os.path.join(project_dir, f"{prefix}_{phase}.csv")
    tr = make_trk(f"{prefix}_{phase}", logfile)
    import time as _t
    st = start(tr); t0 = _t.time(); res = fn(); t1 = _t.time()
    em = stop(tr, st); ekwh, co2 = unpack(em)
    if ekwh == 0.0:
        ek = read_energy(logfile)
        if ek: ekwh = ek
    return {"phase": phase, "time_s": t1-t0, "energy_kwh": ekwh, "co2_kg": co2}, res

In [ ]:
# ---------- Eval-Config ----------
MODELS = [
    ("bigscience/bloom-560m", "bloom560m", "b560"),
    ("bigscience/bloom-3b",   "bloom3b",   "b3b"),
]
EVAL = {
    "max_new_tokens": 32,
    "ppl":  {"name":"wikitext","config":"wikitext-2-raw-v1","split":"test[:1%]"},
    "bleu": {"name":"wmt14","config":"de-en","split":"test[:32]"},
}
PROMPTS = [
    "Schreibe einen kurzen Absatz über nachhaltige KI.",
    "Erkläre in einfachen Worten, was Quantisierung in neuronalen Netzen ist.",
    "Nenne drei Vorteile von Mixture-of-Experts-Modellen."
]
bleu_metric = evaluate.load("sacrebleu")

# Eval-Schutz
MAX_LEN_CAP = 256  # ggf. 192/128 für noch weniger VRAM

def autocast_ctx():
    return torch.autocast(device_type="cuda", dtype=torch.float16) if torch.cuda.is_available() else nullcontext()

def safe_max_len(tok, model, fallback=2048, upper=100000):
    cand = getattr(tok, "model_max_length", None)
    if isinstance(cand, int) and 0 < cand < upper: return cand
    cand = getattr(getattr(model, "config", None), "max_position_embeddings", None)
    if isinstance(cand, int) and 0 < cand < upper: return cand
    return fallback

def capture_memory():
    ram = psutil.Process().memory_info().rss
    valloc = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    vres  = torch.cuda.memory_reserved()  if torch.cuda.is_available() else 0
    return ram, valloc, vres

def bytes_to_gb(b): return float(b)/(1024**3)

In [ ]:
# ---------- Laden + Hardware-Tweaks (BLOOM benötigt eager attention) ----------
from transformers import AutoTokenizer, AutoModelForCausalLM
def load_model_fp16_optimized(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    tok.padding_side = "left"; tok.pad_token = tok.eos_token

    if torch.cuda.is_available():
        torch.cuda.empty_cache(); gc.collect()

    # WICHTIG: BLOOM → attn_implementation="eager" (SDPA nicht unterstützt)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
    )
    try:
        model.config.use_cache = True
    except Exception:
        pass

    if APPLY_TORCH_COMPILE and hasattr(torch, "compile"):
        try:
            model = torch.compile(model, mode="reduce-overhead", fullgraph=False)
            print("[HW] torch.compile aktiv (reduce-overhead).")
        except Exception as e:
            print("[HW] torch.compile nicht möglich → normal weiter:", repr(e))

    model.eval()
    return tok, model

In [ ]:
# ---------- Utility: Pinned Memory ----------
def _pin_and_move(batch, device):
    """
    Erwartet ein Dict mit Tensors; pint Host-Speicher und verschiebt non_blocking.
    """
    out = {}
    for k, v in batch.items():
        if isinstance(v, torch.Tensor):
            if PINNED_MEM and v.device.type == "cpu":
                try:
                    v = v.pin_memory()
                except Exception:
                    pass
            out[k] = v.to(device, non_blocking=NON_BLOCKING)
        else:
            out[k] = v
    return out

def get_model_device(model):
    try:
        return next(model.parameters()).device
    except StopIteration:
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ---------- Evaluation (VARIANTE 2: exakte Tokenzählung pro Beispiel) ----------
def simple_generate(model, tok, prompts, max_new_tokens=32):
    dev = get_model_device(model)
    ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
    # Batch alle Prompts zusammen -> höherer Durchsatz
    enc = tok(prompts, return_tensors="pt", truncation=True, max_length=ml, padding=True)
    enc = _pin_and_move(enc, dev)
    room = ml - enc["input_ids"].shape[1]
    cur_new = max(1, min(max_new_tokens, int(room)))
    with torch.inference_mode(), autocast_ctx():
        out = model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)

    texts, total_gen_tokens = [], 0
    max_input_len = int(enc["input_ids"].shape[1])
    pad_id = tok.pad_token_id
    eos_id = tok.eos_token_id

    # genaue Zählung: tokens in der generierten Region [max_input_len: ] pro Beispiel,
    # EOS wird als generierter Token gezählt (auch wenn pad_id == eos_id)
    for i in range(out.size(0)):
        seq = out[i]
        gen_slice = seq[max_input_len:]  # nur generierter Teil (gepadde Länge)
        gen_i = 0
        for t in gen_slice.tolist():
            if t == eos_id:
                gen_i += 1  # EOS mitzählen
                break
            if (pad_id is not None) and (t == pad_id):
                break
            gen_i += 1
        total_gen_tokens += gen_i
        texts.append(tok.decode(seq, skip_special_tokens=True))

    return texts, total_gen_tokens

def eval_perplexity(model, tok, ds_cfg):
    dev = get_model_device(model)
    ds = load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
    losses = []
    with torch.inference_mode(), autocast_ctx():
        for t in ds["text"]:
            if not isinstance(t, str) or len(t.strip()) < 4: continue
            enc = tok(t, return_tensors="pt", truncation=True, max_length=ml)
            enc = _pin_and_move(enc, dev)
            out = model(enc["input_ids"], labels=enc["input_ids"])
            losses.append(float(out.loss.detach().cpu()))
    return math.exp(np.mean(losses)) if losses else None

def eval_bleu_llm(model, tok, ds_cfg, max_new_tokens=32, batch_size=8):
    dev = get_model_device(model)
    ds = load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
    preds, refs = [], []
    with torch.inference_mode(), autocast_ctx():
        batch_prompts, batch_refs = [], []
        for ex in ds:
            de, en = ex["translation"]["de"], ex["translation"]["en"]
            prompt = f"Translate to English:\nGerman: {de}\nEnglish:"
            batch_prompts.append(prompt); batch_refs.append(en)
            if len(batch_prompts) >= batch_size:
                enc = tok(batch_prompts, return_tensors="pt", truncation=True, max_length=ml, padding=True)
                enc = _pin_and_move(enc, dev)
                room = ml - enc["input_ids"].shape[1]
                cur_new = max(1, min(max_new_tokens, int(room)))
                out = model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)
                for i in range(out.size(0)):
                    gen = tok.decode(out[i], skip_special_tokens=True)
                    hyp = gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
                    preds.append(hyp); refs.append([batch_refs[i]])
                batch_prompts, batch_refs = [], []
        if batch_prompts:
            enc = tok(batch_prompts, return_tensors="pt", truncation=True, max_length=ml, padding=True)
            enc = _pin_and_move(enc, dev)
            room = ml - enc["input_ids"].shape[1]
            cur_new = max(1, min(max_new_tokens, int(room)))
            out = model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)
            for i in range(out.size(0)):
                gen = tok.decode(out[i], skip_special_tokens=True)
                hyp = gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
                preds.append(hyp); refs.append([batch_refs[i]])
    return float(bleu_metric.compute(predictions=preds, references=refs)["score"])

In [ ]:
# ---------- Run pro Modell ----------
def run_once(model_id: str, alias: str):
    print(f"\n### Starte Hardware-Optimierung: {alias} ({model_id})")

    # Warmup + Laden mit HW-Tweaks
    def _do_load():
        tok, model = load_model_fp16_optimized(model_id)
        # kurze Warmup-Token (batch=2), damit Kernels greifen
        dev = get_model_device(model); ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
        wenc = tok(["Warmup token 1", "Warmup token 2"], return_tensors="pt", truncation=True, max_length=ml, padding=True)
        wenc = _pin_and_move(wenc, dev)
        with torch.inference_mode(), autocast_ctx():
            _ = model.generate(**wenc, max_new_tokens=1, do_sample=False, pad_token_id=tok.eos_token_id)
        return tok, model
    m_warm, (tok, model) = measure("warmup", _do_load, f"{RESULT_BASENAME}_{alias}")

    # GEN (batched)
    m_gen, (samples, n_tok) = measure("gen",
        lambda: simple_generate(model, tok, PROMPTS, EVAL["max_new_tokens"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    # PPL
    m_ppl, ppl = measure("ppl",
        lambda: eval_perplexity(model, tok, EVAL["ppl"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    # BLEU (mini-batched)
    m_bleu, bleu = measure("bleu",
        lambda: eval_bleu_llm(model, tok, EVAL["bleu"], EVAL["max_new_tokens"], batch_size=8),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    # Gesamt und "steady-state" (ohne Warmup)
    total_time   = m_warm["time_s"] + m_gen["time_s"] + m_ppl["time_s"] + m_bleu["time_s"]
    total_energy = m_warm["energy_kwh"] + m_gen["energy_kwh"] + m_ppl["energy_kwh"] + m_bleu["energy_kwh"]
    total_co2    = m_warm["co2_kg"]     + m_gen["co2_kg"]     + m_ppl["co2_kg"]     + m_bleu["co2_kg"]

    steady_time   = m_gen["time_s"] + m_ppl["time_s"] + m_bleu["time_s"]
    steady_energy = m_gen["energy_kwh"] + m_ppl["energy_kwh"] + m_bleu["energy_kwh"]
    steady_co2    = m_gen["co2_kg"]     + m_ppl["co2_kg"]     + m_bleu["co2_kg"]

    ram, valloc, vres = capture_memory()

    per_phase = [m_warm, m_gen, m_ppl, m_bleu]
    for p in per_phase:
        p["alias"] = alias
        p["model_id"] = model_id

    row = dict(
        model_id=model_id, alias=alias, precision=f"fp16 (eager, compile={APPLY_TORCH_COMPILE})",
        time_s=total_time, energy_kwh=total_energy, co2_kg=total_co2,
        steady_time_s=steady_time, steady_energy_kwh=steady_energy, steady_co2_kg=steady_co2,
        kg_per_kwh=(total_co2/total_energy) if total_energy else None,
        tokens_out=int(n_tok), ppl=ppl, bleu=bleu,
        ram_GB=bytes_to_gb(ram), vram_alloc_GB=bytes_to_gb(valloc), vram_reserved_GB=bytes_to_gb(vres),
        notes=f"attn=eager; compile={APPLY_TORCH_COMPILE}; pinned={PINNED_MEM}; nb={NON_BLOCKING}; MAX_LEN_CAP={MAX_LEN_CAP}"
    )

    # Samples speichern
    samples_path = os.path.join(project_dir, f"{RESULT_BASENAME}_samples_{alias}.txt")
    with open(samples_path, "w", encoding="utf-8") as f:
        for i, txt in enumerate(samples, 1):
            f.write(f"--- Beispiel {i} ({alias}) ---\n{txt}\n\n")
    print("Beispiele gespeichert:", samples_path)

    return row, per_phase

In [ ]:
# ---------- Ausführen & Speichern ----------
all_rows, all_phases = [], []
for mid, long_name, alias in MODELS:
    row, phases = run_once(mid, alias)
    all_rows.append(row)
    all_phases.extend(phases)

df  = pd.DataFrame(all_rows).sort_values("alias").reset_index(drop=True)
dfp = pd.DataFrame(all_phases)
dfp["wh_total"] = dfp["energy_kwh"] * 1000.0
dfp["phase"] = pd.Categorical(dfp["phase"], categories=["warmup","gen","ppl","bleu"], ordered=True)

out_csv = os.path.join(project_dir, f"{RESULT_BASENAME}_results.csv")
df.to_csv(out_csv, index=False)

out_phase_csv = os.path.join(project_dir, f"{RESULT_BASENAME}_per_phase.csv")
dfp.to_csv(out_phase_csv, index=False)

print("\nErgebnisse (gesamt):")
print(df)
print("Gespeichert (gesamt):", out_csv)

print("\nPer-Phase Übersicht:")
print(dfp[["alias","phase","time_s","energy_kwh","co2_kg"]].sort_values(["alias","phase"]))
print("Gespeichert (per Phase):", out_phase_csv)

[codecarbon WARNING @ 14:31:53] Multiple instances of codecarbon are allowed to run at the same time.



### Starte Hardware-Optimierung: b560 (bigscience/bloom-560m)


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

Beispiele gespeichert: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_hardware/bloom_hardware_samples_b3b.txt

Ergebnisse (gesamt):
                model_id alias                    precision     time_s  \
0    bigscience/bloom-3b   b3b  fp16 (eager, compile=False)  45.777770   
1  bigscience/bloom-560m  b560  fp16 (eager, compile=False)  48.226489   

   energy_kwh   co2_kg  steady_time_s  steady_energy_kwh  steady_co2_kg  \
0    0.001468  0.00048      12.156497           0.000451       0.000001   
1    0.001443  0.00016      37.375516           0.001135       0.000015   

   kg_per_kwh  tokens_out         ppl       bleu    ram_GB  vram_alloc_GB  \
0    0.326743          96  222.807096  10.054131  3.251450       5.601610   
1    0.110682          96  316.370096   5.069542  2.621548       1.049553   

   vram_reserved_GB                                              notes  
0          6.798828  attn=eager; compile=False; pinned=True; nb=Tru...  
1          1.062500  